In [1]:
import os
import json
import pandas
import numpy as np
import pickle
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import callbacks
import tensorflow as tf
from keras.regularizers import l2

% matplotlib inline

Using TensorFlow backend.


In [2]:
Ned = []
NRd = []
MRd = []
fck = []
As = []
bh = []
a = []
xu = []
l0 = []
listdir = os.listdir("../../data_set/set_2")


for file in listdir:
    f = open("../../data_set/set_2/" + file)
    d = json.loads(f.read())
    f.close()

    for o in d:
        Ned.append(o["Ned"])
        #NRd.append(o["NRd"])
        MRd.append(o["MRd"])
        fck.append(o["fck"])
        As.append(o["As"])
        bh.append(o["bh"])
        #a.append(o["a"])
        #xu.append(o["xu"])
        l0.append(o["l0"])

rho = []
for i in range(len(As)):
    rho.append(As[i] / bh[i]**2)

In [3]:
df = pandas.DataFrame({
        "l0": l0,
        "Ned": Ned,
        "fck": fck,
        "bh": bh,
        "rho": rho,
        "MRd": MRd
    }
    )
print(df.shape)
df = df[df["fck"] == 20]
print(df.shape)
df.head(5)

(816823, 6)
(244748, 6)


,MRd,Ned,bh,fck,l0,rho
231,43927563,1000000,280,20,1047.632313,0.005740
232,60545085,1000000,280,20,1109.253971,0.008291
233,64683817,1000000,280,20,1124.131709,0.008929
234,68836536,1000000,280,20,1138.815097,0.009566
235,72999386,1000000,280,20,1153.311559,0.010204


In [35]:
feat_df = df[[ "MRd", "Ned", "l0"]]
res_df = df[["bh"]]

scale_feat = preprocessing.StandardScaler()
scale_feat.fit(feat_df)
print(scale_feat.transform(feat_df))

scale_res = preprocessing.StandardScaler()
scale_res.fit(res_df)
df = df.sample(frac=1)


[[-0.21739594 -0.39648395  0.02269942]
 [-0.38946697 -1.77223312  1.44006448]
 [ 0.23468431 -1.17695704  0.63530684]
 ..., 
 [ 2.74999477  0.67501301  0.79290612]
 [-0.97080925  0.17233542 -0.96609084]
 [-0.24254868 -0.6610511  -0.2071171 ]]


In [53]:
split = int(200e3)
df_train = df[:split]
df_test = df[split:]
df_train.head(5)

print(df_train.shape, df_test.shape)
df_train.head(15)

(200000, 6) (44748, 6)


,MRd,Ned,bh,fck,l0,rho
389166,527210915,2040000,600,20,3098.258410,0.002500
136355,832482694,1340000,640,20,4714.249392,0.005615
220831,387364776,1560000,540,20,2875.616437,0.002572
80354,304143213,1190000,520,20,3006.848346,0.002034
767371,406386420,640000,680,20,6993.698483,0.001946
153078,678129464,1380000,660,20,4662.795820,0.003329
700165,722613214,320000,660,20,10401.161367,0.006198
290291,226690312,1760000,400,20,1745.120029,0.009375
586542,1128937006,2630000,780,20,4638.838134,0.002712
16351,477403910,1030000,640,20,4935.822452,0.002319


In [54]:
train_feat = scale_feat.transform(df_train[["MRd", "Ned", "l0"]])
train_label = df_train["bh"].as_matrix()
test_feat = scale_feat.transform(df_test[["MRd", "Ned", "l0"]])
test_label = df_test["bh"].as_matrix()

print("Training features:\n MRd, Ned, l0\n")
print(train_feat[:5])

print("\nTraining labels:\n bh\n")
print(train_label[:5])


print("\nTraining data:", train_feat.shape, train_label.shape)
print("\nTesting data:", test_feat.shape, test_label.shape)



Training features:
 MRd, Ned, l0

[[-0.17103328  0.54272943 -0.38177279]
 [ 0.87424962 -0.38325559  0.43708833]
 [-0.64988127 -0.09223173 -0.49459078]
 [-0.93484072 -0.58168095 -0.42809244]
 [-0.58474915 -1.30924061  1.59213948]]

Training labels:
 bh

[600 640 540 520 680]

Training data: (200000, 3) (200000,)

Testing data: (44748, 3) (44748,)


In [55]:
X = train_feat
Y = train_label

m = Sequential([
        Dense(20, input_dim=3, W_regularizer=l2(0.01)),
        Activation("relu"),
        Dense(1)
    ])
m.compile(optimizer="adam",
         loss="mse",
         metrics=["mean_squared_error"])

es = callbacks.EarlyStopping()

m = Sequential()
m.add(Dense(32, input_dim=3)
m.add(Activation("relu")
m.add(Dense(1))

In [ ]:
with tf.device('/gpu:0'):
    m.fit(X, Y, nb_epoch=10, batch_size=32, validation_data=(test_feat, test_label), callbacks=[es])

Train on 200000 samples, validate on 44748 samples
Epoch 1/10
200000/200000 [==============================] - 6s - loss: 132153.0297 - mean_squared_error: 132148.9790 - val_loss: 11332.0277 - val_mean_squared_error: 11325.9738

In [51]:
input = {
    "MRd": [0.04],
    "Ned": [2500e3],
    "l0": [4e3],
}

for i in input:
    input[i].append(0)

inp = pandas.DataFrame(input)
print("INPUT\n",inp.head(1))

x = scale_feat.transform(inp[["MRd", "Ned", "l0"]])
a = m.predict(x)
print("\n\nOUTPUT:\n", a[0])

h = a[0]
ac = h**2
# #as_ = ac * input["rho"][0]


# n = input["Ned"][0] / (ac * 20 + as_ * 435)
# ylim = 10.8 / n**0.5

# print("ylim", ylim)
# print("y", input["l0"][0] / ( h / 3.46 ))

INPUT
     MRd        Ned      l0
0  0.04  2500000.0  4000.0


OUTPUT:
 [ 785.47113037]


In [52]:
# Test
test_feat = scale_feat.transform(df_test[["MRd", "Ned", "l0"]])
test_res = df_test[["bh"]].as_matrix()

pred = m.predict(test_feat)

for i in range(10):
    print("Prediction :", int(pred[i]), "Computed: ", test_res[i][0], "Deviation: %s%%" % (round(1 - (pred[i] / test_res[i])[0], 2) * 100)) 

Prediction : 271 Computed:  240 Deviation: -13.0%
Prediction : 769 Computed:  780 Deviation: 1.0%
Prediction : 462 Computed:  420 Deviation: -10.0%
Prediction : 704 Computed:  720 Deviation: 2.0%
Prediction : 375 Computed:  380 Deviation: 1.0%
Prediction : 418 Computed:  420 Deviation: 0.0%
Prediction : 277 Computed:  260 Deviation: -7.0%
Prediction : 464 Computed:  460 Deviation: -1.0%
Prediction : 336 Computed:  320 Deviation: -5.0%
Prediction : 426 Computed:  460 Deviation: 7.0%


In [49]:
pred = m.predict(test_feat)
dev = 0
for i in range(pred.shape[0]):
    dev += abs(1 - (pred[i] / test_res[i])[0])

print("Total accuracy %s%%" % (100 - round(dev / pred.shape[0] * 100, 2)))

Total accuracy 96.85%
